In [4]:
# %pip install -q anywidget ipywidgets>=8
# (Colab only; safe no-op elsewhere)
# from google.colab import output; output.enable_custom_widget_manager()

import anywidget, traitlets as T
from IPython.display import display

class TextEchoWidget(anywidget.AnyWidget):
    _esm = r"""
    function render({ model, el }) {
      const div = document.createElement('div');
      div.textContent = "Waiting for Python message…";
      el.appendChild(div);

      // Receive Python -> JS messages (JSON or string)
      model.on('msg:custom', (msg /* JSON|string */, buffers) => {
        div.textContent = (typeof msg === 'string')
          ? msg
          : JSON.stringify(msg, null, 2);
        console.log(msg);
      });
    }
    export default { render };
    """

w = TextEchoWidget()
display(w)

# Send anything JSON-able (or a plain string); it will show up in the text block
w.send({"hello": "world", "n": 123})
w.send("just a string")




TextEchoWidget()

In [5]:
%%timeit -n 10
w.send(b"just a string heehe")


39.7 μs ± 16.3 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
import av

In [16]:
av.codecs_available

{'012v',
 '4xm',
 '8bps',
 '8svx_exp',
 '8svx_fib',
 'a64multi',
 'a64multi5',
 'aac',
 'aac_fixed',
 'aac_latm',
 'aasc',
 'ac3',
 'ac3_fixed',
 'acelp.kelvin',
 'adpcm_4xm',
 'adpcm_adx',
 'adpcm_afc',
 'adpcm_agm',
 'adpcm_aica',
 'adpcm_argo',
 'adpcm_ct',
 'adpcm_dtk',
 'adpcm_ea',
 'adpcm_ea_maxis_xa',
 'adpcm_ea_r1',
 'adpcm_ea_r2',
 'adpcm_ea_r3',
 'adpcm_ea_xas',
 'adpcm_ima_acorn',
 'adpcm_ima_alp',
 'adpcm_ima_amv',
 'adpcm_ima_apc',
 'adpcm_ima_apm',
 'adpcm_ima_cunning',
 'adpcm_ima_dat4',
 'adpcm_ima_dk3',
 'adpcm_ima_dk4',
 'adpcm_ima_ea_eacs',
 'adpcm_ima_ea_sead',
 'adpcm_ima_iss',
 'adpcm_ima_moflex',
 'adpcm_ima_mtf',
 'adpcm_ima_oki',
 'adpcm_ima_qt',
 'adpcm_ima_rad',
 'adpcm_ima_smjpeg',
 'adpcm_ima_ssi',
 'adpcm_ima_wav',
 'adpcm_ima_ws',
 'adpcm_ms',
 'adpcm_mtaf',
 'adpcm_psx',
 'adpcm_sbpro_2',
 'adpcm_sbpro_3',
 'adpcm_sbpro_4',
 'adpcm_swf',
 'adpcm_thp',
 'adpcm_thp_le',
 'adpcm_vima',
 'adpcm_xa',
 'adpcm_xmd',
 'adpcm_yamaha',
 'adpcm_zork',
 'agm',
 'aic

In [ ]:
import anywidget


# TODO ref https://riju.github.io/web-codecs/#examples
# TODO ref https://w3c.github.io/webcodecs/samples/video-decode-display/worker.js
class ViewerWidget(anywidget.AnyWidget):
    # TODO
    _esm = r"""
    // TODO
    import * as msgpackr from "https://esm.sh/msgpackr@1.10.2";

    export default {
        render({ model, el }) {
            const canvas = document.createElement('canvas');
            canvas.width = 640; canvas.height = 360;
            // TODO
            el.appendChild(canvas);

            const canvasContext = canvas.getContext('bitmaprenderer');

            const decoder = new VideoDecoder({
                output: async (frame) => {
                    // console.log(frame);
                    try {
                        canvasContext.transferFromImageBitmap(
                            await createImageBitmap(frame)
                        );
                    } finally {
                        frame.close();
                    }
                },
                error: (e) => console.error('[decoder error]', e),
            });

            model.on('msg:custom', (msg, buffers) => {
                // TODO
                try {
                    for (const buffer of buffers) {
                        const [command, options] = msgpackr.unpack(buffer.buffer);
                        switch (command) {
                            case "configure":
                                decoder.configure({
                                    codec: options.codec,
                                    description: options.description ?? undefined,
                                    // TODO
                                    hardwareAcceleration: 'prefer-hardware',
                                    optimizeForLatency: true,
                                    latencyMode: 'realtime',
                                });
                                break;
                            case "decode":
                                const chunk = new EncodedVideoChunk({
                                    type: options.type,
                                    timestamp: options.timestamp,
                                    data: options.data,
                                    transfer: [options.data.buffer],
                                });
                                decoder.decode(chunk);
                                break;
                            default:
                                break;
                        }
                    }
                } catch (e) {console.error(e);}
            });
        }
    };
    """


viewer_widget = ViewerWidget()
display(viewer_widget)

ViewerWidget()

In [ ]:
import av
import fractions

enc = av.codec.CodecContext.create("h264", "w")

width: int = 640
height: int = 320
fps: int = 30
bitrate: str = "3M"

enc.width = width
enc.height = height
enc.time_base = fractions.Fraction(1, fps)   # pts units
enc.framerate = fractions.Fraction(fps, 1)
# TODO https://developer.mozilla.org/en-US/docs/Web/Media/Guides/Formats/codecs_parameter#container_format_mime_types
enc.pix_fmt = "yuv420p"
enc.options = {
    # "preset": "veryfast",
    "tune": "zerolatency",
    # "b": bitrate,
    "g": str(fps),           # keyframe every ~1s
    "keyint_min": str(fps),
    "rc_lookahead": "0",
    "annexb": "1",
    "repeat_headers": "1",   # SPS/PPS before each IDR simplifies bootstrap
    "profile": "baseline",
    "level": "3.1",
}
enc.open()

In [22]:
# TODO ref: https://developer.chrome.com/docs/web-platform/best-practices/webcodecs

In [23]:
import msgpack

todo_pkt = msgpack.packb(["configure", {
    "codec": "avc1.42E01E",
    "description": bytes(enc.extradata) if enc.extradata is not None else None,
}])

viewer_widget.send(
    "todo",
    [todo_pkt],
)


In [ ]:

import msgpack

import numpy as np

for _ in range(10):
    img_bgr = np.random.randint(0, 256, (640, 360, 3), dtype=np.uint8)

    vf = av.VideoFrame.from_ndarray(img_bgr, format="bgr24").reformat(format="yuv420p")
    [pkt] = enc.encode(vf)

    pts = pkt.pts if pkt.pts is not None else 0
    ts_us = int((pts * pkt.time_base) * 1_000_000)

    todo_pkt = msgpack.packb(["decode", {
        "type": "key" if pkt.is_keyframe else "delta",
        "timestamp": ts_us,
        "data": bytes(pkt),
    }])

    viewer_widget.send(
        "todo",
        [todo_pkt],
    )

    import time
    time.sleep(1 / 30)

In [ ]:
%%html

<canvas id="c" style="width: 100%; height: 100px; border: 1px solid black;" contenteditable tabindex="0"></canvas>
<p id="demo"></p>

<script type="module">

    /*
    interface ViewerOptions {
        comm?: ViewerCommunication;
    }

    function useViewer(canvas: HTMLCanvasElement, options: ViewerOptions) {
        // TODO ...
    }
    
    */

    function useViewer(canvas, options) {
        const comm = options?.comm;
        const abortController = new AbortController();

        // NOTE this will be automatically called the first time
        const resizeObserver = new ResizeObserver(() => {
            const scale = window.devicePixelRatio ?? 1;
            const width = Math.floor(canvas.clientWidth  * scale);
            const height = Math.floor(canvas.clientHeight * scale);

            // NOTE setting .width or .height clears canvas buffer so use sparingly
            // NOTE this also prevents the remote from echoing
            if (canvas.width === width && canvas.height === height) 
                return;

            canvas.width = width; 
            canvas.height = height;            

            comm?.emit({
                id: "win:conf",
                // TODO should std use scaled size or unscaled?
                size: [canvas.width, canvas.height],
                scale: window.devicePixelRatio ?? 1,
            });
        });
        resizeObserver.observe(canvas);
        abortController.signal.addEventListener(
            "abort", 
            () => resizeObserver.disconnect(), 
            { once: true },
        );

        if (true) {
            const bitmapContext = canvas.getContext("bitmaprenderer");
            const decoder = new VideoDecoder({
                output: async (frame) => {
                    // TODO rm
                    // console.log(frame);
                    try {
                        bitmapContext.transferFromImageBitmap(
                            await createImageBitmap(frame)
                        );
                    } finally {
                        frame.close();
                    }
                },
                // TODO
                error: (e) => console.error('[decoder error]', e),
            });

            // TODO abort signal
            const videoHandler = (event) => {
                switch (event.id) {
                    case "vid:conf":
                        // TODO
                        decoder.configure({
                            codec: event.codec,
                            description: event.description ?? undefined,
                            // TODO
                            hardwareAcceleration: 'prefer-hardware',
                            optimizeForLatency: true,
                            latencyMode: 'realtime',
                        });
                        break;
                    case "vid:dec":
                        const chunk = new EncodedVideoChunk({
                            type: event.type,
                            timestamp: event.timestamp,
                            data: event.data,
                            transfer: [event.data.buffer],
                        });
                        decoder.decode(chunk);
                        break;
                }
            };
            comm?.on?.(videoHandler);
            abortController.signal.addEventListener(
                "abort", 
                () => comm?.off?.(videoHandler), 
                { once: true },
            );
        }

        // TODO

        if (true) {
            canvas.addEventListener(
                "click", 
                (event) => {
                    event.preventDefault();
                    canvas.focus();
                    comm?.emit({id: "win:focus"});
                }, 
                { signal: abortController.signal, passive: false },
            );            
        }

        // TODO

        if (true) {
            canvas.addEventListener(
                "pointermove", 
                (event) => {
                    event.preventDefault();
                    comm?.emit({
                        id: "inp:pt",
                        pos: [event.offsetX, event.offsetY],
                    });
                }, 
                { signal: abortController.signal, passive: false },
            );

            canvas.addEventListener(
                "pointerdown", 
                (event) => {
                    event.preventDefault();            
                    comm?.emit({
                        id: "inp:pt",
                        action: "down",
                        button: event.button,
                    });
                }, 
                { signal: abortController.signal, passive: false },
            );

            canvas.addEventListener(
                "pointerup", 
                (event) => {
                    event.preventDefault();            
                    comm?.emit({
                        id: "inp:pt",
                        action: "up",
                        button: event.button,
                    });
                }, 
                { signal: abortController.signal, passive: false },
            );

            canvas.addEventListener(
                "contextmenu", 
                (event) => event.preventDefault(), 
                { signal: abortController.signal, passive: false },
            );            
        }

        if (true) {
            canvas.addEventListener(
                "wheel", 
                (event) => {
                    event.preventDefault();
                    comm?.emit({
                        id: "inp:whl",
                        deltaPos: [event.deltaX, event.deltaY, event.deltaZ],
                    });
                }, 
                { signal: abortController.signal, passive: false },
            );
        }

        if (true) {
            canvas.addEventListener(
                "keydown", 
                (event) => {
                    event.preventDefault();
                    comm?.emit({
                        id: "inp:kb",
                        action: "down",
                        key: event.key,
                    });
                }, 
                { signal: abortController.signal, passive: false },
            );

            canvas.addEventListener(
                "keyup", 
                (event) => {
                    event.preventDefault();
                    comm?.emit({
                        id: "inp:kb",
                        action: "up",
                        key: event.key,
                    });
                }, 
                { signal: abortController.signal, passive: false },
            );
        }

        // TODO clipboard

        if (true) {
            canvas.addEventListener(
                "focus",
                (event) => {
                    // TODO
                },
                { signal: abortController.signal, passive: false },
            );

            canvas.addEventListener(
                "blur",
                (event) => {
                    // TODO
                },
                { signal: abortController.signal, passive: false },
            );
        }

        return {
            disconnect() {
                abortController.signal.abort();
            }
        };
    }


    const c = document.getElementById('c');  
    const demo = document.getElementById('demo');

    const comm = {
        emit(event) {
            demo.textContent = JSON.stringify(event);
        }
    };

    useViewer(c, { comm });


    // TODO
    /*
    import * as msgpackr from "https://esm.sh/msgpackr@1.10.2";

    function useWidgetComm(model) {

        const listenerHandlers = new Map();

        // TODO cleanup
        return {
            emit(event) {
                model.send(null, null, [msgpackr.pack(event)]);
            },
            on(listener) {
                // TODO
                var handler;
                if (listenerHandlers.has(listener)) {
                    handler = listenerHandlers.get(listener);
                } else {
                    handler = (msg, buffers) => {
                        for (const buffer of buffers) {
                            listener(msgpackr.unpack(buffer.buffer));
                        }
                    };
                    listenerHandlers.set(listener, handler);
                }
                model.on("msg:custom", handler);
            },
            off(listener) {
                if (listenerHandlers.has(listener)) {
                    model.off("msg:custom", listenerHandlers.get(listener));
                    listenerHandlers.delete(listener);
                }
            },
        };

        

    }
    */

</script>

In [ ]:
import anywidget
import msgpack


# TODO ref https://riju.github.io/web-codecs/#examples
# TODO ref https://w3c.github.io/webcodecs/samples/video-decode-display/worker.js
class ViewerWidget(anywidget.AnyWidget):
    # TODO
    pass


class ViewerWidgetCommunication:
    def __init__(self, widget: ViewerWidget):
        self.widget = widget
        self._listener_handlers = dict()

    # TODO
    def emit(self, event: ...):
        self.widget.send(None, [msgpack.packb(event)])

    def on(self, listener: ...):
        if listener in self._listener_handlers:
            handler = self._listener_handlers[listener]
        else:
            def handler(widget, msg, buffers):
                for buffer in buffers:
                    listener(msgpack.unpackb(buffer))            
            self._listener_handlers[listener] = handler
        self.widget.on_msg(handler, remove=False)

    def off(self, listener: ...):
        if listener in self._listener_handlers:
            self.widget.on_msg(self._listener_handlers[listener], remove=True)
            self._listener_handlers.pop(listener)
